# Student Loan Risk with Deep Learning

In [30]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [36]:
# Check the credit_ranking value counts
loans_df.credit_ranking.value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [37]:
# Define the target set y using the credit_ranking column
y = loans_df.credit_ranking.values

# Display a sample of y
y

array([0, 0, 0, ..., 1, 0, 1])

In [6]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop('credit_ranking', axis=1)

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [31]:
# Create a StandardScaler instance
import sklearn as skl
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
display(X_train_scaled)
display(X_test_scaled)

array([[-0.56794375, -1.18088385, -0.19241296, ...,  0.73249574,
        -0.22553894, -0.31915735],
       [-0.00940983,  0.80065564, -0.04123135, ..., -0.03016065,
        -0.75844691, -1.15783431],
       [ 2.33643265, -0.84597577,  1.67216021, ..., -1.36480934,
         0.42579302,  0.05358797],
       ...,
       [-0.9589175 ,  1.07974571, -0.89792713, ...,  0.22405814,
        -0.9952949 ,  0.33314696],
       [-0.62379715,  0.54947458, -1.35147195, ...,  0.35116754,
        -0.46238693, -1.34420697],
       [ 0.43741731, -0.73433974,  1.16822152, ..., -0.66570765,
         0.18894504,  1.91731456]])

array([[ 0.26985713, -0.67852173,  1.87373569, ...,  1.43159743,
         0.01130905,  0.05358797],
       [ 0.21400374,  0.54947458,  0.05955639, ..., -0.34793415,
        -0.16632694, -0.22597102],
       [ 1.16351141, -1.06924782,  1.57137247, ..., -0.79281704,
         0.24815703,  0.7990786 ],
       ...,
       [-0.62379715, -0.95761179,  0.96664604, ...,  0.54183164,
         1.13633698,  0.51951962],
       [-1.51745143,  0.46574755, -1.35147195, ...,  1.68581622,
        -0.81765891, -0.50553001],
       [ 1.94545891, -1.12506584,  1.11782765, ..., -0.09371535,
         0.60342901,  0.23996063]])

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [38]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
print(input_nodes)


11


In [42]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 18

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 12

# Define the number of neurons in the output layer
nerouns_output_layer = 1

In [43]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_nodes, activation="relu"))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=nerouns_output_layer, activation="sigmoid")) 

In [44]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 18)             │           216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 12)             │           228 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            13 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 457 (1.79 KB)

 Trainable params: 457 (1.79 KB)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [45]:
# Compile the Sequential model
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 807us/step - accuracy: 0.5235 - loss: 0.6802 
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 786us/step - accuracy: 0.6392 - loss: 0.6530
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step - accuracy: 0.7029 - loss: 0.6057
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step - accuracy: 0.7417 - loss: 0.5815
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - accuracy: 0.7462 - loss: 0.5525
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step - accuracy: 0.7677 - loss: 0.5243
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step - accuracy: 0.7563 - loss: 0.5376
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - accuracy: 0.7288 - loss: 0.5384
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step - accuracy: 0.7496 - loss: 0.5187
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 760us/step - accuracy: 0.7656 - loss: 0.5121
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step - accuracy: 0.7824 - loss: 0.4846
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/ste

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [47]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - 8ms/step - accuracy: 0.7425 - loss: 0.5186
Loss: 0.5186363458633423, Accuracy: 0.7425000071525574


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [48]:
# Set the model's file path
file_path = Path("saved_models/student_loans.keras")

# Export your model to a keras file
nn_model.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [49]:
# Set the model's file path
file_path_load = Path('saved_models/student_loans.keras')

# Load the model to a new object
nm_imported = tf.keras.models.load_model(file_path_load)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [50]:
# Make predictions with the test data
predictions = nm_imported.predict(X_test_scaled)

# Display a sample of the predictions
predictions

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


array([[0.38256076],
       [0.3766263 ],
       [0.800949  ],
       [0.7228809 ],
       [0.971564  ],
       [0.7711432 ],
       [0.8732173 ],
       [0.04983753],
       [0.70866436],
       [0.21734017],
       [0.9299578 ],
       [0.22756596],
       [0.40453956],
       [0.94738835],
       [0.63336253],
       [0.4430255 ],
       [0.89373064],
       [0.28416798],
       [0.45619345],
       [0.41279632],
       [0.6193454 ],
       [0.7928469 ],
       [0.21070154],
       [0.9167288 ],
       [0.35785422],
       [0.95785147],
       [0.86131144],
       [0.5685562 ],
       [0.18150873],
       [0.64038634],
       [0.547174  ],
       [0.9691112 ],
       [0.19722328],
       [0.96290606],
       [0.2996181 ],
       [0.40304977],
       [0.14645615],
       [0.37519762],
       [0.96825814],
       [0.11584468],
       [0.9299829 ],
       [0.08293407],
       [0.05624055],
       [0.9399664 ],
       [0.0942426 ],
       [0.7977911 ],
       [0.11590892],
       [0.272

In [51]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame({'Predictions': predictions.flatten()})
predictions_df['Predictions'] = predictions_df['Predictions'].round()
predictions_df


,Predictions
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [52]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df['Predictions']))

              precision    recall  f1-score   support

           0       0.71      0.78      0.74       188
           1       0.78      0.71      0.75       212

    accuracy                           0.74       400
   macro avg       0.74      0.74      0.74       400
weighted avg       0.75      0.74      0.74       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. To build a recommendation system for loan options, we need to collect a ton of data that looks at their education background, financial profile and goals. Depending on the details of a student's background can better help understand the loan option as well as assess how likely the loan is to be paid back. For example education background is important because a student is more likely to get a high paying job and hold a job depending on the level of completed education. Also knowing financial data like credit score or history gives insight into the students repayment patterns to further assess whether or not the loan will be paid back on time.

**2. Based on the data chosen, I would choose content-based filtering because loan options would be based on the student's profile and preferences. It wouldn't require a large user base or historical ratings and you can easily understand why a loan is recommended to a particular student.

**3. One real world challenge of building a student loan recommendation system is the quality of the data. Students may have incomplete profiles or they don't enter all of their personal information to assess the best loan. Loans vary by each vendor so the products themselves lack any standardized format.

Another challenge is there might be some bias in the training data. There could occur some unintended algorithmic discrimination like race or gender. The fairness of these loans might be imbalanced in fairness to marginalized groups.